# Processing/Regridding CESM inputs and outputs
Regridding SE and FV output and emissions
Regridding CESM ocean T grids / U grids
Calculate budgets, totals as in AMWG/Chemistry diagnostics
Creating timeseries from averages over regions
